<a href="https://colab.research.google.com/github/gumdropsteve/silent-disco/blob/master/bsql_table_from_pandas_cuDF_and_BlazingSQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Data Check / Download
The below cell will check if you have the data for this demo and download it for you if not.

In [1]:
%%time 
import os
import urllib

# check for existance of data directory
if not os.path.exists('data/'):
    # it does not, so let us know & create it
    print('creating data directory')
    os.system('mkdir data')

# base url to raw data on GitHub 
base_url = 'https://raw.githubusercontent.com/gumdropsteve/turbo-telegram/master/data/'

# download jan-mar 2015 taxi data
for month in ['jan', 'feb', 'march']:
  fn = f'nyc_taxi_{month}15.csv'
  # check if we already have the file
  if not os.path.isfile(f'data/{fn}'):
      # we don't let me know we're downloading it now
      print(f'Downloading {base_url + fn} to data/{fn}')
      # download file
      urllib.request.urlretrieve(base_url + fn, f'data/{fn}')
  # we already have data
  else:
      # let us know
      print(f'{fn} already downloaded')
    
# identify current working directory & wildcard path to data
cwd = os.getcwd()

CPU times: user 459 ms, sys: 499 ms, total: 959 ms
Wall time: 3.56 s


## Imports

In [2]:
%%time
import cudf
import pandas as pd
from blazingsql import BlazingContext
# connect to BlazingSQL
bc = BlazingContext()

BlazingContext ready
CPU times: user 2.44 s, sys: 779 ms, total: 3.21 s
Wall time: 2.58 s


### pandas -> BlazingSQL

In [3]:
%%time
# create pandas DataFrame
df = pd.read_csv('data/nyc_taxi_jan15.csv')

# create BlazingSQL table from pandas DataFrame
bc.create_table('pd_taxi', df)

# query BlazingSQL table & display last 3 rows
bc.sql('SELECT * FROM pd_taxi').tail(3)

CPU times: user 2.42 s, sys: 277 ms, total: 2.7 s
Wall time: 2.25 s


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_x,pickup_y,RateCodeID,store_and_fwd_flag,dropoff_x,dropoff_y,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
697498,2,2015-01-31 20:46:12,2015-01-31 20:57:49,1,2.31,-8.237828e+06,4.972035e+06,1,N,-8.239360e+06,4.969136e+06,2,10.0,0.5,0.5,0.0,0.0,0.3,11.3
697499,1,2015-01-31 21:04:48,2015-01-31 21:20:53,1,3.10,-8.233061e+06,4.978176e+06,1,N,-8.236266e+06,4.972510e+06,1,13.0,0.5,0.5,1.0,0.0,0.3,15.3
697500,1,2015-01-31 20:52:51,2015-01-31 21:20:06,1,6.90,-8.234850e+06,4.981938e+06,1,N,-8.237785e+06,4.971117e+06,1,25.5,0.5,0.5,1.2,0.0,0.3,28.0


### cuDF -> BlazingSQL

In [4]:
%%time
# create pandas DataFrame
df = pd.read_csv('data/nyc_taxi_jan15.csv')

# create BlazingSQL table from pandas DataFrame
bc.create_table('pd_taxi', df)

# query BlazingSQL table & display last 3 rows
bc.sql('SELECT * FROM pd_taxi').tail(3)

CPU times: user 1.54 s, sys: 264 ms, total: 1.8 s
Wall time: 1.78 s


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_x,pickup_y,RateCodeID,store_and_fwd_flag,dropoff_x,dropoff_y,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
697498,2,2015-01-31 20:46:12,2015-01-31 20:57:49,1,2.31,-8.237828e+06,4.972035e+06,1,N,-8.239360e+06,4.969136e+06,2,10.0,0.5,0.5,0.0,0.0,0.3,11.3
697499,1,2015-01-31 21:04:48,2015-01-31 21:20:53,1,3.10,-8.233061e+06,4.978176e+06,1,N,-8.236266e+06,4.972510e+06,1,13.0,0.5,0.5,1.0,0.0,0.3,15.3
697500,1,2015-01-31 20:52:51,2015-01-31 21:20:06,1,6.90,-8.234850e+06,4.981938e+06,1,N,-8.237785e+06,4.971117e+06,1,25.5,0.5,0.5,1.2,0.0,0.3,28.0


### BlazingSQL -> BlazingSQL

In [5]:
%%time
# create BlazingSQL table from 3 CSV files jan-mar 2015
bc.create_table('q1_2015', f'{cwd}/data/nyc_taxi_*.csv', header=0)

# create BlazingSQL table from BlazingSQL query results
bc.create_table('single_10mile', bc.sql('SELECT * FROM q1_2015 WHERE trip_distance > 10 AND passenger_count = 1'))

# pull payment info from january query results table
bc.sql('SELECT payment_type, fare_amount, extra, mta_tax, tip_amount, tolls_amount, total_amount FROM single_10mile')

CPU times: user 1.23 s, sys: 546 ms, total: 1.78 s
Wall time: 893 ms


,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,total_amount
0,2,36.0,0.0,0.5,0.00,5.33,42.13
1,1,34.0,0.5,0.5,7.05,0.00,42.35
2,1,35.0,0.0,0.5,8.20,5.33,49.33
3,1,52.0,0.0,0.5,11.63,5.33,69.76
4,1,33.5,0.5,0.5,5.00,5.54,45.34
...,...,...,...,...,...,...,...
15759,2,33.0,0.5,0.5,0.00,0.00,34.30
15760,1,43.0,0.0,0.5,8.76,0.00,52.56
15761,1,30.0,0.5,0.5,7.33,5.33,43.96
15762,1,36.0,0.0,0.5,8.43,5.33,50.56
